### Lendo dados da camada landing-zone e salvando no formato Delta na camada bronze 

In [0]:
from pyspark.sql.functions import current_timestamp, lit

storageAccountName = "satcseguroimoveis"
tables = ["apolice_cobertura", "cobertura", "sinistro", "pagamento", "apolice", "avaliacao", "imovel", "cliente"]


for table in tables:
    remote_table = spark.read.option("infeschema", "true").option("header", "true").parquet(f"/mnt/{storageAccountName}/landing-zone/{table}")
    remote_table = remote_table.withColumn("dt_insert_bronze", current_timestamp()).withColumn("filename", lit(table))
    remote_table.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/{table}")

### Verificando se os dados foram salvados corretamente com os metadados

In [0]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/bronze/")) 

path,name,size,modificationTime
dbfs:/mnt/satcseguroimoveis/bronze/apolice/,apolice/,0,0
dbfs:/mnt/satcseguroimoveis/bronze/apolice_cobertura/,apolice_cobertura/,0,0
dbfs:/mnt/satcseguroimoveis/bronze/avaliacao/,avaliacao/,0,0
dbfs:/mnt/satcseguroimoveis/bronze/cliente/,cliente/,0,0
dbfs:/mnt/satcseguroimoveis/bronze/cobertura/,cobertura/,0,0
dbfs:/mnt/satcseguroimoveis/bronze/imovel/,imovel/,0,0
dbfs:/mnt/satcseguroimoveis/bronze/pagamento/,pagamento/,0,0
dbfs:/mnt/satcseguroimoveis/bronze/sinistro/,sinistro/,0,0


### Verificando se a tabela é gerenciada ou não pelo Databricks

In [0]:
df_cliente = spark.read.format("delta").load('/mnt/satcseguroimoveis/bronze/cliente')
df_cliente.show(10)

+----------+-------------------+----------+--------------------+-------------------+--------------------+--------------------+--------+
|id_cliente|               nome|   dt_nasc|            endereco|           telefone|               email|    dt_insert_bronze|filename|
+----------+-------------------+----------+--------------------+-------------------+--------------------+--------------------+--------+
|         1|  Dr. Rhavi Barbosa|1992-01-19|Rodovia de Melo, ...|   +55 41 5800 2366|dr.rhavibarbosa@r...|2024-06-27 03:05:...| cliente|
|         2|          Evelyn Sá|1986-08-09|Setor Ravi Lucca ...|   +55 31 9959-5996|   evelynsá@melo.com|2024-06-27 03:05:...| cliente|
|         3|  Sra. Maysa Barros|1950-03-25|Passarela Vasconc...|+55 (011) 7038-6784|sra.maysabarros@o...|2024-06-27 03:05:...| cliente|
|         4|  Maria Flor Barros|1966-08-19|Praça de Rodrigue...|       51 2645 3083|mariaflorbarros@g...|2024-06-27 03:05:...| cliente|
|         5|Maria Vitória Silva|1936-04-09|Resid